## Necessary Imports

In [1]:
import pandas as pd
import numpy as np
import spacy
import numpy as np
import pandas as pd
from nltk.corpus import stopwords
from tqdm import tqdm
import re
import os
import json
from collections import defaultdict

## A. Dataset and Preprocessing:


In [7]:
train_path = 'train.csv'
test_path = 'test.csv'

train_df = pd.read_csv(train_path)
test_df = pd.read_csv(test_path)

training_data = []
test_data = []
validation_data = []

In [ ]:
def create_datasets(train_df, test_df):
    shuffled = np.random.permutation(len(train_df)) #to randomly get 100 indices for validation dataset creation
    validation_indices = shuffled[:100]
    
    training_data = []
    validation_data = []
    test_data = []
    
    for index, row in train_df.iterrows():
        if index not in validation_indices:
            training_data.append(row['text'])
        else:
            validation_data.append(row['text'])
            
    for index, row in test_df.iterrows():
        test_data.append(row['text'])
                    
    return [training_data, test_data, validation_data]

training_data, test_data, validation_data = create_datasets(train_df=train_df, test_df=test_df)

In [ ]:
def pre_process_data(spacy_model, data):
    
    processed_list = []
    stop_words = set(stopwords.words("english"))
    
    for text in tqdm(data, desc = 'preprocessing dataset...'):
        
        text = text.lower()         
        text = re.sub(r'[^a-zA-Z\s]', '', text)         # subsitituting any non english alphabet with ''
        text = ' '.join(text.split())               #joining all the words to form a sentence

        tokens = spacy_model(text)              #tokenization part, i.e lemmatization is done and sentence split into words
        
        processed_tokens = [token.lemma_ for token in tokens if token.text.isalpha() and token.text not in stop_words and token.text.strip() != ""]                 # only adding those tokens that have text as alphabets and no punctuation or stopwords in them
        
        processed_list.append(processed_tokens)
        
    return processed_list

def clean_data(training_data, test_data, validation_data):
    
    spacy_model = spacy.load("en_core_web_sm")
        
    
    curr_dirr = os.getcwd()
    data_path = os.path.join(curr_dirr, 'processed_data_with_lemmatizer.json')
    
    print(f'saving processed_data to {data_path}')
    
    training_data = pre_process_data(spacy_model=spacy_model, data = training_data)
    test_data = pre_process_data(spacy_model=spacy_model, data = test_data)
    validation_data = pre_process_data(spacy_model=spacy_model, data = validation_data)
    
    data_dict = {'training_data' : training_data, 'test_data' : test_data, 'validation_data' : validation_data  }
    
    data_list = [data_dict]
    
    with open(data_path, 'w') as file:
        json.dump(data_list, file, indent = 4)
        
    return

clean_data(training_data=training_data, test_data=test_data, validation_data=validation_data)

preprocessing dataset...: 100%|██████████| 100/100 [00:32<00:00,  3.10it/s]


saving processed_data to /Users/aryan/Desktop/KGP_MnC/6thSEM/NLP/processed_data_with_lemmatizer.json


## B. Estimation Using Maximum Likelihood:


In [ ]:
def create_ngram_model(data, n=1):
    total_ngrams = 0
    n_gram_counts = defaultdict(int)
    total_articles = len(data)
    n_1gram_counts = defaultdict(int)
    ngram_unique_article_counts = defaultdict(int)
    
    for article in tqdm(data, desc='processing articles..'):
        ngram_seen_in_article_set = set()  
        for i in range(len(article) - n + 1):
            ngram = tuple(article[i:i + n])  
            n_gram_counts[ngram] += 1
            total_ngrams += 1
            if n > 1:  # (n-1)-gram for bigrams, trigrams, etc.
                n_1gram_counts[ngram[:-1]] += 1  # Count the (n-1)-gram (prefix of the n-gram)
            
            if ngram not in ngram_seen_in_article_set:
                ngram_unique_article_counts[ngram] += 1  # count the occurrence of an ngram only once per article
                ngram_seen_in_article_set.add(ngram)  # to mark the ngram as seen in the article
        
    # to filter the articles that appear in atleast 1% of all the articles
    vocabulary = {ngram: n_gram_counts[ngram] for ngram, count in ngram_unique_article_counts.items() if count >= 0.01*total_articles}
    V = len(vocabulary)            
    
    # MLE Estimation and laplace smoothing for all ngrams
    ngram_probabilities = {}
    for ngram, count in tqdm(n_gram_counts.items(), desc = 'calculating probabilities....'):
        if n == 1: 
            ngram_probabilities[ngram] = (count + 1)/ (total_ngrams + V)  
        else:  # For bigrams, trigrams, etc.
            n_1gram = ngram[:-1]  # the n-1 gram for the bigram, trigram etc
            n_1gram_count = n_1gram_counts[n_1gram]  # no of time the n-1 gram appears
            ngram_probabilities[ngram] = (count+1) / (n_1gram_count + V)  # laplace smoothing with MLE
    
    return (ngram_probabilities, n_1gram_counts, V)

In [5]:
data_path = 'processed_data_with_lemmatizer.json'

with open(data_path, 'r', encoding="utf-8") as file:
    data = json.load(file)
    training_data = data[0]['training_data']
    validation_data = data[0]['validation_data']
    test_data = data[0]['test_data']
    
trigram_model = create_ngram_model(data=training_data, n = 3)
bigram_model =  create_ngram_model(data=training_data, n = 2)   
unigram_model = create_ngram_model(data=training_data, n = 1)


calculating probabilities....: 100%|██████████| 449270/449270 [00:00<00:00, 3089270.95it/s]


In [6]:
unigram_probabilities, unigram_n_1_counts, unigram_vocab_size = unigram_model
bigram_probabilities, bigram_n_1_counts, bigram_vocab_size = bigram_model
trigram_probabilities, trigram_n_1_counts, trigram_vocab_size = trigram_model

## C. Evaluating an n-Gram Model using Perplexity: 


In [ ]:
def compute_perplexity(test_data, ngram_probabilities, n_1gram_counts, n=1, K=1, V=1):
    mean_perplexity = 0

    for article in tqdm(test_data, desc = 'calculating perplexity for articles....'):
        log_sum_of_prob = 0
        T = len(article)
        
        for i in range(len(article) - n + 1):
            ngram = tuple(article[i:i + n])           #get ngram
            n_1gram = ngram[:-1] if n > 1 else None  # get the n-1 gram prefix of ngram

            # get probability correspoinding to the ngram or apply laplace smoothing if missing
            prob = ngram_probabilities.get(ngram, K / (n_1gram_counts.get(n_1gram, 0) + K * V))
                
            log_sum_of_prob += np.log(prob)

        # Cacluate perplexity for the article
        perplexity = np.exp(-log_sum_of_prob / T)
        mean_perplexity+=perplexity

    # Caclulate overall perplexity as the average across all articles
    mean_perplexity /= len(test_data)
    return mean_perplexity

perplexity_unigram = compute_perplexity(test_data, unigram_probabilities, unigram_n_1_counts, n = 1, K = 1, V = unigram_vocab_size)
perplexity_bigram = compute_perplexity(test_data, bigram_probabilities, bigram_n_1_counts, n = 2, K = 1, V = bigram_vocab_size)
perplexity_trigram = compute_perplexity(test_data, trigram_probabilities, trigram_n_1_counts, n = 3, K = 1, V = trigram_vocab_size)

print(f'perplexity for unigram = {perplexity_unigram}')
print()
print(f'perplexity for bigram = {perplexity_bigram}')
print()
print(f'perplexity for trigram = {perplexity_trigram}')

calculating perplexity for articles....: 100%|██████████| 100/100 [00:02<00:00, 47.03it/s]

perplexity for unigram = 6439.512962763581

perplexity for bigram = 1237.20352695804

perplexity for trigram = 552.9354736657151


## D. Interpolation Model: 


In [ ]:
def compute_interpolated_probabilities(trigram_probabilities, bigram_probabilities, unigram_probabilities, lambda1, lambda2, lambda3):
    
    interpolated_probabilities = dict.fromkeys(tqdm(trigram_probabilities, desc='Creating dict'), 0)

    for trigram, trigram_probability in tqdm(trigram_probabilities.items(), desc = 'interpolating probabilities....'):
        
        interpolated_probabilities[trigram] = lambda1*trigram_probability + lambda2*bigram_probabilities[trigram[1:]] + lambda3*unigram_probabilities[trigram[2:]]
        
    return interpolated_probabilities

In [ ]:
interpolated_probabilities = compute_interpolated_probabilities(
        trigram_probabilities=trigram_probabilities,
    bigram_probabilities = bigram_probabilities, 
unigram_probabilities=unigram_probabilities, 
    lambda1=0.7,
        lambda2=0.3,
            lambda3= 0
        )
print(f'best perplexity using interpolation model : {compute_perplexity(test_data, interpolated_probabilities, trigram_n_1_counts, n = 3, K = 1, V = trigram_vocab_size)}\nlambda1 : 0.7, lambda2  : 0.3, lambda3 : 0')

calculating perplexity for articles....: 100%|██████████| 100/100 [00:01<00:00, 72.97it/s] 

best perplexity using interpolation model : 531.9069643347829
lambda1 : 0.7, lambda2  : 0.3, lambda3 : 0


In [ ]:
grid_space = np.linspace(0, 1, 11)

def grid_search(grid_space):
    
    best_lambdas = [1, 0, 0]
    
    best_perplexity = 1e8

    for lambda1 in grid_space:
        for lambda2 in grid_space:
            
            if (lambda2 + lambda1 > 1):
                break
            else:
                
                lambda3 = 1-lambda1-lambda2
                print(f'checking lambda1 = {lambda1}, lambda2 = {lambda2}, lambda3 = {lambda3}')
                
                interpolated_probabilities = compute_interpolated_probabilities(trigram_probabilities=trigram_probabilities, unigram_probabilities = unigram_probabilities, bigram_probabilities=bigram_probabilities, lambda1=lambda1,lambda2=lambda2,lambda3= lambda3)
                
                current_perplexity = compute_perplexity(validation_data, interpolated_probabilities, trigram_n_1_counts, n = 3, K = 1, V = trigram_vocab_size)
                
                print("current perplexity : ", current_perplexity, [lambda1, lambda2, lambda3])
                print()
                
                if current_perplexity < best_perplexity:
                    best_perplexity = current_perplexity
                    best_lambdas = [lambda1, lambda2, lambda3]
                    print("best perplexity : ",best_perplexity, best_lambdas)
                    print()
                    
    return (best_perplexity, best_lambdas)
    
best_perplexity, best_lambdas = grid_search(grid_space)
print(best_perplexity, best_lambdas)

checking lambda1 = 0.0, lambda2 = 0.0, lambda3 = 1.0


calculating perplexity for articles....: 100%|██████████| 100/100 [00:00<00:00, 101.73it/s]


current perplexity :  1278.715756638718 [0.0, 0.0, 1.0]

best perplexity :  1278.715756638718 [0.0, 0.0, 1.0]

checking lambda1 = 0.0, lambda2 = 0.1, lambda3 = 0.9


calculating perplexity for articles....: 100%|██████████| 100/100 [00:01<00:00, 67.94it/s]


current perplexity :  851.2818698324786 [0.0, 0.1, 0.9]

best perplexity :  851.2818698324786 [0.0, 0.1, 0.9]

checking lambda1 = 0.0, lambda2 = 0.2, lambda3 = 0.8


...

current perplexity :  559.5803196715702 [0.9, 0.0, 0.09999999999999998]

checking lambda1 = 0.9, lambda2 = 0.1, lambda3 = -2.7755575615628914e-17


calculating perplexity for articles....: 100%|██████████| 100/100 [00:00<00:00, 100.47it/s]


current perplexity :  535.0532618833385 [0.9, 0.1, -2.7755575615628914e-17]

checking lambda1 = 1.0, lambda2 = 0.0, lambda3 = 0.0


calculating perplexity for articles....: 100%|██████████| 100/100 [00:00<00:00, 100.54it/s]

current perplexity :  549.6792421554559 [1.0, 0.0, 0.0]

529.3690305594669 [0.7000000000000001, 0.30000000000000004, -1.1102230246251565e-16]


## below is for the validation dataset

In [ ]:
print(best_perplexity, best_lambdas)

529.3690305594669 [0.7000000000000001, 0.30000000000000004, -1.1102230246251565e-16]


## below is for test dataset 

In [13]:
interpolated_probabilities = compute_interpolated_probabilities(unigram_probabilities, bigram_probabilities, trigram_probabilities, lambda1=0.7, lambda2=0.3, lambda3=0)
print(compute_perplexity(test_data, interpolated_probabilities, trigram_n_1_counts, n = 3, K = 1, V=trigram_vocab_size))

calculating perplexity for articles....: 100%|██████████| 100/100 [00:00<00:00, 114.31it/s]

531.9069643347831
